<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltaMergeSchema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaOptimize") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040920 sha256=ff4f21312f8279234a3719354a399aac0eb28c71d7ad20a7b174e2d7cadcb92b
  Stored in directory: /root/.cache/pip/wheels/81/6c/a6/af459ed3106da8f1171a265c2982425fe390aa418a47d2a8b4
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [ ]:
!rm -rf /content/sample_data/merge_schema

In [ ]:
from delta.tables import *

DeltaTable.create(spark)\
    .tableName('employee')\
    .addColumn('empid',"INT")\
    .addColumn('empname','STRING')\
    .addColumn('gender','STRING')\
    .addColumn('dept','STRING')\
    .addColumn('salary','INT')\
    .property('description','Table for mergeSchema Demo')\
    .location('/content/sample_data/merge_schema').execute()


In [ ]:
spark.sql('select * from employee').show()

+-----+-------+------+----+------+
|empid|empname|gender|dept|salary|
+-----+-------+------+----+------+
+-----+-------+------+----+------+



In [ ]:
spark.sql('insert into employee values(1,"Sagar","M","Billing",30000);')

DataFrame[]

In [ ]:
spark.sql('select * from employee').show()

+-----+-------+------+-------+------+
|empid|empname|gender|   dept|salary|
+-----+-------+------+-------+------+
|    1|  Sagar|     M|Billing| 30000|
+-----+-------+------+-------+------+



In [ ]:
import pandas as pd

# Provided data
employee_data = [{'empid': 2, 'empname': 'Revanth', 'gender': 'M', 'dept': 'HR', 'salary': 25000, 'additional_col': 'test'}]

# Create a pandas DataFrame
df_pandas = pd.DataFrame(employee_data)

# Display the DataFrame
print("Pandas DataFrame created:")
display(df_pandas)

# Save the DataFrame to a CSV file
df_pandas.to_csv("employee_data.csv", index=False)

print("\nDataFrame saved to employee_data.csv")

Pandas DataFrame created:


,empid,empname,gender,dept,salary,additional_col
0,2,Revanth,M,HR,25000,test



DataFrame saved to employee_data.csv


In [ ]:
# Read the CSV file into a Spark DataFrame
spark_df = spark.read.csv("employee_data.csv", header=True, inferSchema=True)

# Write the Spark DataFrame to a Delta Lake table
delta_table_path = "/content/sample_data/merge_schema"
spark_df.write.format("delta").mode("append").save(delta_table_path)

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 196ed77e-85eb-4f0d-8a12-63f89cc0177e).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- empid: integer (nullable = true)
-- empname: string (nullable = true)
-- gender: string (nullable = true)
-- dept: string (nullable = true)
-- salary: integer (nullable = true)


Data schema:
root
-- empid: integer (nullable = true)
-- empname: string (nullable = true)
-- gender: string (nullable = true)
-- dept: string (nullable = true)
-- salary: integer (nullable = true)
-- additional_col: string (nullable = true)

         

As can be seen in the error message, when try to pass the csv file to load into already created table, it is showing error of schema mismatch, to overcome this issue, there is a option of "mergeSchema" while writing data to table, which will help in updating the schema of table and load the data, without any issue to pipeline, and for all the previous data it would populate for the extra column that is added to table.

In [ ]:
spark_df.write.format("delta").option("mergeSchema","true").mode("append").save(delta_table_path)

In [ ]:
spark.sql('select * from employee').show()

+-----+-------+------+-------+------+--------------+
|empid|empname|gender|   dept|salary|additional_col|
+-----+-------+------+-------+------+--------------+
|    2|Revanth|     M|     HR| 25000|          test|
|    1|  Sagar|     M|Billing| 30000|          null|
+-----+-------+------+-------+------+--------------+



In [ ]:
spark.table('employee').show()

+-----+-------+------+-------+------+--------------+
|empid|empname|gender|   dept|salary|additional_col|
+-----+-------+------+-------+------+--------------+
|    2|Revanth|     M|     HR| 25000|          test|
|    1|  Sagar|     M|Billing| 30000|          null|
+-----+-------+------+-------+------+--------------+



**END OF CODE**